In [ ]:
import pandas as pd
benign_df = pd.read_csv("benign_domain_list.csv", header=None)
malicious_df = pd.read_csv("Malicious_URLs.csv")
mixed_df = pd.read_csv("mixed_domain_list.csv", header=None)
tranco_df = pd.read_csv("tranco_full_list_for_ranking.csv", header=None)

In [ ]:
benign_df

In [ ]:
malicious_df

In [ ]:
mixed_df

In [ ]:
tranco_df

In [36]:
from gettext import install

import pip


pip install newspaper3k

### Defining methods block
from urllib.parse import urlparse

def calculate_percentage_digits(domain):
    digits_count = sum(c.isdigit() for c in domain)
    total_chars = len(domain)
    percentage_digits = digits_count / total_chars * 100
    return percentage_digits

def get_fqdn(url):
    parsed_url = urlparse(url)
    return parsed_url.netloc

def count_unique_chars(domain):
    unique_chars = set(domain)
    return len(unique_chars) - 1     ### -1 because it counts the '.' in the FQDN

def count_hyphens(domain):
    hyphens_count = sum(c == '-' for c in domain)
    return hyphens_count

def count_dots(domain):
    dot_count = sum(c == '.' for c in domain)
    return dot_count

In [ ]:
benign_df['Domain Length'] = benign_df[0].str.len()
benign_df['Digit Percentage'] = benign_df[0].apply(calculate_percentage_digits)
benign_df['Unique Character Count'] = benign_df[0].apply(count_unique_chars)
benign_df['Hypen Count'] = benign_df[0].apply(count_hyphens)
benign_df['Dot Count'] = benign_df[0].apply(count_dots)
benign_df

In [ ]:
malicious_df['Domain'] = malicious_df['url'].apply(get_fqdn)
malicious_df['Domain Length'] = malicious_df['Domain'].str.len()
malicious_df['Digit Percentage'] = malicious_df['Domain'].apply(calculate_percentage_digits)
malicious_df['Digit Count'] = malicious_df['Domain'].apply(count_unique_chars)
malicious_df['Hyphen Count'] = malicious_df['Domain'].apply(count_hyphens)
malicious_df['Dot Count'] = malicious_df['Domain'].apply(count_dots)
malicious_df

In [ ]:

mixed_df['Domain Length'] = mixed_df[0].str.len()
mixed_df['Digit Percentage'] = mixed_df[0].apply(calculate_percentage_digits)
mixed_df['Digit Count'] = mixed_df[0].apply(count_unique_chars)
mixed_df['Hypen Count'] = mixed_df[0].apply(count_hyphens)
mixed_df['Dot Count'] = mixed_df[0].apply(count_dots)
mixed_df

In [ ]:
tranco_df['Domain Length'] = tranco_df[1].str.len()
tranco_df['Digit Percentage'] = tranco_df[1].apply(calculate_percentage_digits)
tranco_df['Digit Count'] = tranco_df[1].apply(count_unique_chars)
tranco_df['Hypen Count'] = tranco_df[1].apply(count_hyphens)
tranco_df['Dot Count'] = tranco_df[1].apply(count_dots)
tranco_df